In [1]:
import pandas as pd
import numpy as np

stn=pd.read_csv('./csv/stn.csv')
weather=pd.read_csv('./csv/weather_new.csv')

stn = stn[['STN_ID', 'STN_KO']]

# STN을 기준으로 병합하여 STN_KO를 Weather 데이터에 추가
weather=weather.merge(stn, left_on='STN', right_on='STN_ID', how='left')
weather.head()


,TM,STN,WS_AVG,WR_DAY,WD_MAX,WS_MAX,WS_MAX_TM,WD_INS,WS_INS,WS_INS_TM,...,SD_NEW_TM,SD_MAX,SD_MAX_TM,TE_05,TE_10,TE_15,TE_30,TE_50,STN_ID,STN_KO
0,20140101,90.0,5.4,4691.0,27.0,9.8,2224.0,27.0,18.0,2111.0,...,-9.0,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,90.0,속초
1,20140101,95.0,3.2,2741.0,23.0,7.3,1527.0,23.0,12.5,1611.0,...,-9.0,0.5,1.0,2.2,5.6,-99.0,-99.0,-99.0,95.0,철원
2,20140101,98.0,2.6,2257.0,25.0,7.9,1541.0,25.0,13.7,1259.0,...,-9.0,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,98.0,동두천
3,20140101,99.0,2.0,1742.0,23.0,6.1,1442.0,25.0,10.0,1521.0,...,-9.0,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,99.0,파주
4,20140101,100.0,10.6,9158.0,27.0,15.5,1837.0,27.0,22.7,1935.0,...,-9.0,2.4,0.0,-99.0,-99.0,-99.0,-99.0,-99.0,100.0,대관령


In [3]:
#생산량 데이터 전처리 작업
production=pd.read_csv('./csv/생산량.csv')

# 생산량을 숫자로 변환 (소수점이 포함된 값이 있으므로 이를 처리)
production['생산량'] = pd.to_numeric(production['생산량'], errors='coerce')

# NaN 값 제거
production = production.dropna(subset=['생산량'])

# '면적'이 포함된 행 제거
production2 = production[~production['작물명'].str.contains('면적')]

# '합계'가 포함된 행 제거
production3 = production2[~production2['작물명'].str.contains('합계')]

# '10a당'이 포함된 행 제거
production4 = production3[~production3['작물명'].str.contains('10a당')]

# '류'가 포함된 행 제거
production5 = production4[~production4['작물명'].str.contains('류')]

# '계'가 포함된 행 제거
production_end = production5[~production5['지역'].str.contains('계')]

# '작물명'에서 ':생산량' 제거
production_end['작물명'] = production_end['작물명'].str.replace(':생산량', '')

# '생산량'이 0인 행 제거
production_end = production_end[production_end['생산량'] != 0]

production_end.index = range(1, len(production_end) + 1)
production_end.to_csv('./csv/production.csv', index=False)

production_end

C:\Users\sunk2\AppData\Local\Temp\ipykernel_20748\2474448184.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  production_end['작물명'] = production_end['작물명'].str.replace(':생산량', '')


,생산량,년도,지역,작물명,단위
1,4.0,2017,서울특별시,수박,톤
2,2.0,2018,서울특별시,수박,톤
3,2.0,2019,서울특별시,수박,톤
4,1.0,2020,서울특별시,수박,톤
5,8.0,2021,서울특별시,수박,톤
...,...,...,...,...,...
11566,49.0,2018,제주도,녹두,톤
11567,99.0,2019,제주도,녹두,톤
11568,18.0,2020,제주도,녹두,톤
11569,27.0,2021,제주도,녹두,톤


In [4]:
#광역도시로 세부지역을 매핑하기
mapping = {
    #강원도
    '춘천': '강원도', '원주': '강원도', '강릉': '강원도', '양양': '강원도', '철원': '강원도', '회양': '강원도',
    '이천': '강원도', '삼척': '강원도', '영월': '강원도', '평해': '강원도', '통천': '강원도', '정선': '강원도',
    '고성': '강원도', '간성': '강원도', '평창': '강원도', '금성': '강원도', '울진': '강원도', '흡곡': '강원도',
    '평강': '강원도', '김화': '강원도', '화천': '강원도', '홍천': '강원도', '양구': '강원도', '인제': '강원도',
    '횡성': '강원도', '안협': '강원도', 
    
    # 충청남도
    '천안': '충청남도', '공주': '충청남도', '보령': '충청남도', '아산': '충청남도', '서산': '충청남도', '논산': '충청남도',
    '계룡': '충청남도', '당진': '충청남도', '금산': '충청남도', '부여': '충청남도', '서천': '충청남도', '청양': '충청남도',
    '홍성': '충청남도', '예산': '충청남도', '태안': '충청남도',
    
    # 경기도
    '수원': '경기도', '고양': '경기도', '용인': '경기도', '성남': '경기도', '부천': '경기도', '안산': '경기도', '안양': '경기도',
    '남양주': '경기도', '화성': '경기도', '평택': '경기도', '의정부': '경기도', '파주': '경기도', '시흥': '경기도', '김포': '경기도',
    '광명': '경기도', '광주': '경기도', '군포': '경기도', '오산': '경기도', '이천': '경기도', '양주': '경기도', '구리': '경기도',
    '안성': '경기도', '포천': '경기도', '의왕': '경기도', '하남': '경기도', '여주': '경기도', '양평': '경기도', '동두천': '경기도',
    '과천': '경기도', '가평': '경기도', '연천': '경기도',
    
    # 경상북도
    '포항': '경상북도', '경주': '경상북도', '김천': '경상북도', '안동': '경상북도', '구미': '경상북도', '영주': '경상북도',
    '영천': '경상북도', '상주': '경상북도', '문경': '경상북도', '경산': '경상북도', '군위': '경상북도', '의성': '경상북도',
    '청송': '경상북도', '영양': '경상북도', '영덕': '경상북도', '청도': '경상북도', '고령': '경상북도', '성주': '경상북도',
    '칠곡': '경상북도', '예천': '경상북도', '봉화': '경상북도', '울진': '경상북도', '울릉': '경상북도',

    # 경상남도
    '창원': '경상남도', '진주': '경상남도', '통영': '경상남도', '사천': '경상남도', '김해': '경상남도', '밀양': '경상남도',
    '거제': '경상남도', '양산': '경상남도', '의령': '경상남도', '함안': '경상남도', '창녕': '경상남도', '고성': '경상남도',
    '남해': '경상남도', '하동': '경상남도', '산청': '경상남도', '함양': '경상남도', '거창': '경상남도', '합천': '경상남도',

    # 제주도
    '제주시': '제주도', '서귀포시': '제주도',

    # 전라남도
    '목포': '전라남도', '여수': '전라남도', '순천': '전라남도', '나주': '전라남도', '광양': '전라남도', '담양': '전라남도',
    '곡성': '전라남도', '구례': '전라남도', '고흥': '전라남도', '보성': '전라남도', '화순': '전라남도', '장흥': '전라남도',
    '강진': '전라남도', '해남': '전라남도', '영암': '전라남도', '무안': '전라남도', '함평': '전라남도', '영광': '전라남도',
    '장성': '전라남도', '완도': '전라남도', '진도': '전라남도', '신안': '전라남도',

    # 서울특별시
    '서울': '서울특별시',
    
    # 전라북도
    '전주': '전라북도', '군산': '전라북도', '익산': '전라북도', '정읍': '전라북도', '남원': '전라북도', '김제': '전라북도',
    '완주': '전라북도', '진안': '전라북도', '무주': '전라북도', '장수': '전라북도', '임실': '전라북도', '순창': '전라북도',
    '고창': '전라북도', '부안': '전라북도'
    
    
}

# 날씨 데이터의 세부 지역을 광역 지역으로 매핑
weather['AREA'] = weather['STN_KO'].map(mapping)
weather.head()


,TM,STN,WS_AVG,WR_DAY,WD_MAX,WS_MAX,WS_MAX_TM,WD_INS,WS_INS,WS_INS_TM,...,SD_MAX,SD_MAX_TM,TE_05,TE_10,TE_15,TE_30,TE_50,STN_ID,STN_KO,AREA
0,20140101,90.0,5.4,4691.0,27.0,9.8,2224.0,27.0,18.0,2111.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,90.0,속초,NaN
1,20140101,95.0,3.2,2741.0,23.0,7.3,1527.0,23.0,12.5,1611.0,...,0.5,1.0,2.2,5.6,-99.0,-99.0,-99.0,95.0,철원,강원도
2,20140101,98.0,2.6,2257.0,25.0,7.9,1541.0,25.0,13.7,1259.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,98.0,동두천,경기도
3,20140101,99.0,2.0,1742.0,23.0,6.1,1442.0,25.0,10.0,1521.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,99.0,파주,경기도
4,20140101,100.0,10.6,9158.0,27.0,15.5,1837.0,27.0,22.7,1935.0,...,2.4,0.0,-99.0,-99.0,-99.0,-99.0,-99.0,100.0,대관령,NaN


In [5]:
weather[weather['AREA'].isna()]

,TM,STN,WS_AVG,WR_DAY,WD_MAX,WS_MAX,WS_MAX_TM,WD_INS,WS_INS,WS_INS_TM,...,SD_MAX,SD_MAX_TM,TE_05,TE_10,TE_15,TE_30,TE_50,STN_ID,STN_KO,AREA
0,20140101,90.0,5.4,4691.0,27.0,9.8,2224.0,27.0,18.0,2111.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,90.0,속초,NaN
4,20140101,100.0,10.6,9158.0,27.0,15.5,1837.0,27.0,22.7,1935.0,...,2.4,0.0,-99.0,-99.0,-99.0,-99.0,-99.0,100.0,대관령,NaN
6,20140101,102.0,6.7,5783.0,29.0,11.1,1700.0,29.0,16.1,1619.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,102.0,백령도,NaN
7,20140101,104.0,3.5,3013.0,32.0,7.2,2344.0,29.0,13.3,1946.0,...,-9.0,-9.0,2.9,6.2,8.3,13.5,15.2,104.0,북강릉,NaN
9,20140101,106.0,3.4,2963.0,27.0,7.4,1338.0,27.0,15.6,1330.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,106.0,동해,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363009,20240615,266.0,1.5,1263.0,23.0,3.8,1531.0,23.0,7.0,1623.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,266.0,광양시,NaN
363010,20240615,268.0,1.5,1311.0,32.0,3.7,1352.0,32.0,6.2,1516.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,268.0,진도군,NaN
363014,20240615,276.0,1.1,968.0,11.0,4.0,1523.0,14.0,8.1,1519.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,276.0,청송군,NaN
363019,20240615,283.0,2.0,1692.0,36.0,7.1,1556.0,2.0,13.0,1549.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,283.0,경주시,NaN


In [6]:
# 매핑 딕셔너리에 없는 'STN_KO' 값을 추가
mapping.update({
    '속초': '강원도',
    '동두천': '경기도',
    '파주': '경기도',
    '대관령': '강원도',
    '백령도': '인천광역시',
    '북강릉': '강원도',
    '동해': '강원도',
    '서울': '서울특별시',
    '인천': '인천광역시',
    '울릉도': '경상북도',
    '수원': '경기도',
    '충주': '충청북도',
    '서산': '충청남도',
    '청주': '충청북도',
    '대전': '대전광역시',
    '추풍령': '충청북도',
    '안동': '경상북도',
    '상주': '경상북도',
    '포항': '경상북도',
    '군산': '전라북도',
    '대구': '대구광역시',
    '전주': '전라북도',
    '울산': '울산광역시',
    '창원': '경상남도',
    '광주': '광주광역시',
    '부산': '부산광역시',
    '통영': '경상남도',
    '목포': '전라남도',
    '여수': '전라남도',
    '흑산도': '전라남도',
    '완도': '전라남도',
    '고창': '전라북도',
    '순천': '전라남도',
    '홍성': '충청남도',
    '제주': '제주특별자치도',
    '고산': '제주특별자치도',
    '성산': '제주특별자치도',
    '서귀포': '제주특별자치도',
    '진주': '경상남도',
    '강화': '인천광역시',
    '양평': '경기도',
    '태백': '강원도',
    '정선군': '강원도',
    '제천': '충청북도',
    '보은': '충청북도',
    '천안': '충청남도',
    '보령': '충청남도',
    '부여': '충청남도',
    '금산': '충청남도',
    '부안': '전라북도',
    '임실': '전라북도',
    '정읍': '전라북도',
    '남원': '전라북도',
    '장수': '전라북도',
    '고창군': '전라북도',
    '영광군': '전라남도',
    '김해시': '경상남도',
    '순창군': '전라북도',
    '북창원': '경상남도',
    '양산시': '경상남도',
    '보성군': '전라남도',
    '강진군': '전라남도',
    '장흥': '전라남도',
    '해남': '전라남도',
    '고흥': '전라남도',
    '의령군': '경상남도',
    '함양군': '경상남도',
    '광양시': '전라남도',
    '진도군': '전라남도',
    '봉화': '경상북도',
    '영주': '경상북도',
    '문경': '경상북도',
    '청송군': '경상북도',
    '영덕': '경상북도',
    '의성': '경상북도',
    '구미': '경상북도',
    '영천': '경상북도',
    '경주시': '경상북도',
    '거창': '경상남도',
    '합천': '경상남도',
    '밀양': '경상남도',
    '산청': '경상남도',
    '거제': '경상남도',
    '남해': '경상남도'
})

# 다시 매핑 적용
weather['AREA'] = weather['STN_KO'].map(mapping)

# 'AREA' 컬럼에서 NaN 값인 행을 나타내기
weather[weather['AREA'].isna()]



,TM,STN,WS_AVG,WR_DAY,WD_MAX,WS_MAX,WS_MAX_TM,WD_INS,WS_INS,WS_INS_TM,...,SD_MAX,SD_MAX_TM,TE_05,TE_10,TE_15,TE_30,TE_50,STN_ID,STN_KO,AREA
39,20140101,175.0,5.8,5011.0,32.0,8.9,12.0,32.0,15.6,11.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,NaN,NaN,NaN
40,20140101,176.0,3.6,3075.0,27.0,7.3,1428.0,29.0,12.5,1558.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,NaN,NaN,NaN
132,20140102,175.0,3.0,2604.0,32.0,6.8,47.0,32.0,10.4,41.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,NaN,NaN,NaN
133,20140102,176.0,2.0,1750.0,29.0,5.3,327.0,32.0,7.9,1152.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,NaN,NaN,NaN
225,20140103,175.0,2.0,1757.0,34.0,6.3,2139.0,36.0,8.6,2344.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362929,20240614,296.0,1.9,1645.0,20.0,4.7,1421.0,23.0,7.4,1329.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,NaN,NaN,NaN
362931,20240615,93.0,1.1,963.0,34.0,3.2,1904.0,36.0,4.5,1856.0,...,-9.0,-9.0,26.7,23.0,19.3,14.6,13.1,93.0,북춘천,NaN
362971,20240615,181.0,1.3,1143.0,32.0,5.3,1606.0,32.0,8.5,1556.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,181.0,서청주(예),NaN
362990,20240615,239.0,1.5,1256.0,32.0,5.6,1755.0,34.0,9.5,1750.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,239.0,세종,NaN


In [7]:
# 매핑 딕셔너리에 없는 'STN_KO' 값을 추가
mapping.update({
    '북춘천': '강원도',
    '세종': '세종특별자치시',
    '서청주(예)': '충청북도'
})

# 다시 매핑 적용
weather['AREA'] = weather['STN_KO'].map(mapping)
weather[weather['AREA'].isna()]

,TM,STN,WS_AVG,WR_DAY,WD_MAX,WS_MAX,WS_MAX_TM,WD_INS,WS_INS,WS_INS_TM,...,SD_MAX,SD_MAX_TM,TE_05,TE_10,TE_15,TE_30,TE_50,STN_ID,STN_KO,AREA
39,20140101,175.0,5.8,5011.0,32.0,8.9,12.0,32.0,15.6,11.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,NaN,NaN,NaN
40,20140101,176.0,3.6,3075.0,27.0,7.3,1428.0,29.0,12.5,1558.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,NaN,NaN,NaN
132,20140102,175.0,3.0,2604.0,32.0,6.8,47.0,32.0,10.4,41.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,NaN,NaN,NaN
133,20140102,176.0,2.0,1750.0,29.0,5.3,327.0,32.0,7.9,1152.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,NaN,NaN,NaN
225,20140103,175.0,2.0,1757.0,34.0,6.3,2139.0,36.0,8.6,2344.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362638,20240611,296.0,1.9,1637.0,18.0,5.4,1531.0,18.0,8.5,1457.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,NaN,NaN,NaN
362735,20240612,296.0,1.9,1668.0,20.0,5.1,1605.0,16.0,8.8,1542.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,NaN,NaN,NaN
362832,20240613,296.0,1.6,1347.0,20.0,4.1,1234.0,23.0,7.5,1338.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,NaN,NaN,NaN
362929,20240614,296.0,1.9,1645.0,20.0,4.7,1421.0,23.0,7.4,1329.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,NaN,NaN,NaN


In [10]:
# stn_id가 NaN인 행 삭제
weather = weather.dropna(subset=['STN_ID'])
weather.to_csv('./csv/weather_area_new.csv', index=False)
weather

,TM,STN,WS_AVG,WR_DAY,WD_MAX,WS_MAX,WS_MAX_TM,WD_INS,WS_INS,WS_INS_TM,...,SD_MAX,SD_MAX_TM,TE_05,TE_10,TE_15,TE_30,TE_50,STN_ID,STN_KO,AREA
0,20140101,90.0,5.4,4691.0,27.0,9.8,2224.0,27.0,18.0,2111.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,90.0,속초,강원도
1,20140101,95.0,3.2,2741.0,23.0,7.3,1527.0,23.0,12.5,1611.0,...,0.5,1.0,2.2,5.6,-99.0,-99.0,-99.0,95.0,철원,강원도
2,20140101,98.0,2.6,2257.0,25.0,7.9,1541.0,25.0,13.7,1259.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,98.0,동두천,경기도
3,20140101,99.0,2.0,1742.0,23.0,6.1,1442.0,25.0,10.0,1521.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,99.0,파주,경기도
4,20140101,100.0,10.6,9158.0,27.0,15.5,1837.0,27.0,22.7,1935.0,...,2.4,0.0,-99.0,-99.0,-99.0,-99.0,-99.0,100.0,대관령,강원도
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363021,20240615,285.0,0.7,615.0,18.0,3.8,1832.0,16.0,8.2,1830.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,285.0,합천,경상남도
363022,20240615,288.0,0.8,728.0,23.0,2.9,1637.0,36.0,5.7,1855.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,288.0,밀양,경상남도
363023,20240615,289.0,1.2,1003.0,29.0,3.9,1916.0,32.0,6.9,1942.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,289.0,산청,경상남도
363024,20240615,294.0,1.3,1118.0,20.0,4.4,2011.0,14.0,10.0,2007.0,...,-9.0,-9.0,-99.0,-99.0,-99.0,-99.0,-99.0,294.0,거제,경상남도


In [9]:
weather.columns

Index(['TM', 'STN', 'WS_AVG', 'WR_DAY', 'WD_MAX', 'WS_MAX', 'WS_MAX_TM',
       'WD_INS', 'WS_INS', 'WS_INS_TM', 'TA_AVG', 'TA_MAX', 'TA_MAX_TM',
       'TA_MIN', 'TA_MIN_TM', 'TD_AVG', 'TS_AVG', 'TG_MIN', 'HM_AVG', 'HM_MIN',
       'HM_MIN_TM', 'PV_AVG', 'EV_S', 'EV_L', 'FG_DUR', 'PA_AVG', 'PS_AVG',
       'PS_MAX', 'PS_MAX_TM', 'PS_MIN', 'PS_MIN_TM', 'CA_TOT', 'SS_DAY',
       'SS_DUR', 'SS_CMB', 'SI_DAY', 'SI_60M_MAX', 'SI_60M_MAX_TM', 'RN_DAY',
       'RN_D99', 'RN_DUR', 'RN_60M_MAX', 'RN_60M_MAX_TM', 'RN_10M_MAX',
       'RN_10M_MAX_TM', 'RN_POW_MAX', 'RN_POW_MAX_TM', 'SD_NEW', 'SD_NEW_TM',
       'SD_MAX', 'SD_MAX_TM', 'TE_05', 'TE_10', 'TE_15', 'TE_30', 'TE_50',
       'STN_ID', 'STN_KO', 'AREA'],
      dtype='object')